In [12]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from timeit import default_timer as timer
import illustris_python as il
import physics

#intitial setup
basePath = "../data/tng-100-3/output"
fields = {"stars": ["Coordinates", "Potential", "Masses", "Velocities"],
        "gas": ["Coordinates", "Potential", "Masses", "Velocities"],
        "dm": ["Coordinates", "Potential", "Velocities"]
        }
indices = [544, 953, 1285, 1662]

#indices[0] = 1893

#[544, 953, 1285, 1662]
#[1893, 2164, 3043, 3229, 3403, 3605, 3760] 
#np.genfromtxt("../data/tng-100-3/cutdata/central_ids.txt")
DM_PARTICLE_MASS = 0.03235675 #found in header of snapshot
N = len(indices)
stars = [0]*N
gas = [0]*N
dm = [0]*N
particle_lists = [gas, dm, stars]
group_cat = pd.DataFrame({"id": indices})
#Start timer
start = timer()


In [13]:
#Load all particles
for i in range(N):
    stars[i] = il.pandasformat.dict_to_pandas(il.snapshot.loadSubhalo(basePath, 99, indices[i], 'stars', fields["stars"]))
    gas[i] = il.pandasformat.dict_to_pandas(il.snapshot.loadSubhalo(basePath, 99, indices[i], 'gas', fields["gas"]))
    dm[i] = il.pandasformat.dict_to_pandas(il.snapshot.loadSubhalo(basePath, 99, indices[i], 'dm', fields["dm"]))
    dm[i]["Masses"] = [DM_PARTICLE_MASS]*len(dm[i]["Potential"]) #Add dm masses

subhaloFields = ["SubhaloMass", 'SubhaloMassType', "SubhaloMassInHalfRadType", "SubhaloHalfmassRadType", "SubhaloPos", "SubhaloVel", "SubhaloSpin"]
subhalos = il.groupcat.loadSubhalos(basePath, 99, fields=subhaloFields)
df_subhalos = il.pandasformat.dict_to_pandas(subhalos)

In [14]:
print("Calculating relative positions and radius")
particle_lists, group_cat = physics.properties.relative_pos_radius(particle_lists, N, group_cat)
print("Calculating particle masses")
particle_lists, group_cat = physics.properties.total_mass(particle_lists, N, group_cat)
print("Calculating subhalo velocity")
particle_lists, group_cat = physics.properties.subhalo_velocity(particle_lists, N, group_cat)
print("Calculating relative velocities")
for particle in particle_lists:
    particle = physics.properties.relative_velocities(particle, N, group_cat)
"""
print("Calculating half mass radius")
stars, group_cat = physics.properties.half_mass_radius(stars, N, group_cat, "Stellar")
print("Calculating maximum angular momentum")
group_cat = physics.properties.max_ang_momentum(stars, N, group_cat)
rot_vector = np.transpose(np.array([group_cat["RotationAxisX"], group_cat["RotationAxisY"], group_cat["RotationAxisZ"]]))
print("Rotating subhalos")
particle_lists = physics.geometry.rotate_coordinates(particle_lists, N, group_cat, rot_vector)
"""
#End timer
end = timer()

Calculating relative positions and radius
Calculating particle masses
Calculating subhalo velocity
Calculating relative velocities


In [44]:
def half_mass_radius(particle, N, catalogue, particle_type="Stellar"):
    """
    Calculates half mass radius for a certain particle type and adds it to the group catalogue.
    """
    halfmass_rad = np.zeros(N) #empty list
    mass_key = "SubhaloMass" + particle_type
    rad_key = "SubhaloHalfmassRad" + particle_type
    for i in range(N):
        temp = particle[i].copy(deep=True)
        temp.sort_values(by="r", inplace = True) #sort particles by radius
        temp = temp.reset_index(drop=True)
        temp_mass = 0
        #Start adding the masses of all particles starting with smallest radius
        for j in range(len(temp["r"])):
            #Check if total mass is less than half total particle mass
            if temp_mass < (catalogue[mass_key][i]/2):
                temp_mass = temp_mass + temp["Masses"][j] #Add mass of next particle
            else:
                #Add half mass radius.
                #Some uncertainty on calculation method here, now center of mass between particle j and j-i.
                m1 = temp["Masses"][j-1]
                m2 = temp["Masses"][j]
                M = m1+m2
                halfmass_rad[i] = (m1*temp["r"][j-1] + m2*temp["r"][j])/M
                break  #stop loop
    catalogue[rad_key] = halfmass_rad #save to catalogue
    return particle, catalogue


In [45]:
stars, group_cat = half_mass_radius(stars, N, group_cat, "Stellar")
print(stars[3]["r"][5083])

0 9361 64.72123052750044 64.73282953022934 64.7262886582862
1 10444 78.94338477267323 78.96518300724607 78.95183316992888
2 5144 33.91534743459071 33.923068368323044 33.92006088473392
3 5073 40.65422370017747 40.6627463877318 40.65735770631967
40.95408411044242
5.98725084492989


In [4]:
print("Masses check")
for i in range(3):
    print("Group catalogue: ", df_subhalos["SubhaloMassStellar"][indices[i]], " Snapshot: ", group_cat["SubhaloMassStellar"][i])

Masses check
Group catalogue:  80.1757  Snapshot:  80.17588806152344
Group catalogue:  90.369896  Snapshot:  90.3699951171875
Group catalogue:  43.96099  Snapshot:  43.97459030151367


In [5]:
print("Velocities check")
for i in range(3):
    print(group_cat["SubhaloVelocityX"][i], group_cat["SubhaloVelocityY"][i], group_cat["SubhaloVelocityZ"][i])
    print(df_subhalos["SubhaloVel"][indices[i]])

Velocities check
-175.11496411068748 81.26290417921925 -8.336148174927477
[-175.13785    81.367256   -8.339443]
279.265925652316 -99.84715851410176 82.07070970468858
[279.23257  -99.84879   82.072495]
174.67770423321207 57.30106101855136 170.3806483246425
[174.64732  57.31744 170.41402]


In [46]:
print("Halfmass radius check")
for i in range(3): 
    print("Group catalogue: ", df_subhalos["SubhaloHalfmassRadStellar"][indices[i]], " Snapshot: ", group_cat["SubhaloHalfmassRadStellar"][i])

Halfmass radius check
Group catalogue:  64.70497  Snapshot:  64.7262886582862
Group catalogue:  79.047134  Snapshot:  78.95183316992888
Group catalogue:  33.925236  Snapshot:  33.92006088473392


In [7]:
print("Spin direction  check")
for i in range(3):
    print("Subhalo: ", i)
    s_vec = np.array(df_subhalos["SubhaloSpin"][indices[i]])
    s = np.linalg.norm(s_vec)
    s_dir = s_vec/s
    print("Snapshot : ", rot_vector[i])
    print("catalogue: ", s_dir)

Spin direction  check
Subhalo:  0
Snapshot :  [-0.02886851  0.44894144  0.89309473]
catalogue:  [ 0.21661627 -0.66537654 -0.7143889 ]
Subhalo:  1
Snapshot :  [ 0.59915765 -0.71522081  0.35981843]
catalogue:  [ 0.18161198 -0.07396439 -0.9805847 ]
Subhalo:  2
Snapshot :  [-0.15454508 -0.72022578  0.67630662]
catalogue:  [-0.28974828 -0.75167984  0.5924722 ]


In [8]:
print(gas[0]["Vx"][0])
print(stars[0]["Vx"][0])

-273.7644
-58.701385


In [9]:
#Save all fields for particles in one subhalo for later inspection
g = 0
stars[g].to_pickle("../data/tng-100-3/subhalos/subhalo" + str(indices[g]) +"_stars.pkl")

In [10]:
#Save group catalogue
group_cat.to_pickle("../data/tng-100-3/catalogues/test_cat_01.pkl")

In [11]:

print("Time to process " + str(N) + " Subhalos: ")
print(int(end - start), "Seconds")

Time to process 4 Subhalos: 
37 Seconds
